In [25]:
import tensorflow as tf
import gym
import numpy as np
from tensorflow_probability import distributions as dists
import tensorflow.keras.layers as kl

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

tf.random.set_seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
env = gym.make('Pendulum-v0')

In [3]:
is_continuous = isinstance(env.action_space, gym.spaces.Box)
obs_dim = env.observation_space.shape
act_dim = env.action_space.shape

In [87]:
from rl_agents.ppo.policy import Actor, Critic
from rl_agents.ppo.agent import PPO_Agent

In [88]:
actor = Actor(obs_dim, act_dim, is_continuous)
critic = Critic(obs_dim)

In [89]:
obs = env.reset()
res = actor.predict(obs[None])
print(res)
print(res.shape)

[array([[0.19563769]], dtype=float32), array([[-0.43380758]], dtype=float32), array([[0.01931334]], dtype=float32)]


AttributeError: 'list' object has no attribute 'shape'

In [40]:
agent = PPO_Agent(actor, critic)

In [41]:
obs = env.reset()

act, _, _, _ = agent.action_value(obs[None])

AttributeError: 'Normal' object has no attribute 'type'